# Skyway Network Dataset Generation Notebook
**Paper Title**: Optimizing QoS fulfillment of Drone Services

**Conference Submission**: International Conference on Service-Oriented Computing, 2025

**Purpose**  
This notebook constructs the skyway network used as the operational environment for drone delivery flights, as presented in the paper. It provides all the necessary steps to reproduce the dataset from raw building data of the Sydney CBD area.

**Overview**  
- **Step 1**: Extracts raw building data from OpenStreetMap for the Sydney CBD area. Extracted attributes include:
  - Building height  
  - Building type  
  - GPS coordinates  
  - Building names  
  - Rooftop area  
- **Step 2**: Cleans and preprocesses the data to remove incomplete or invalid entries. 
- **Step 3**: Constructs the skyway network:
  - Nodes serve as the buildings' rooftops. Each node has a varying number of charging and waiting pads based on the usable rooftop area.
  - Line-of-Sight (LoS) segments serve as aerial space for drones to transit between nodes. The area of each skyway segment is defined based on the regulatory measures. 

**Instructions**  
- Run **all cells in order** to fully reproduce the dataset.  

**Dependencies**  
- Python 3.x  
- pandas  
- numpy  
- osmnx  
- shapely  
- networkx  
(Ensure all required packages are installed before execution.)

In [1]:
import requests
import csv
import math
import pandas as pd
import numpy as np
import networkx as nx
import folium
import osmnx as ox
import geopandas as gpd
import concurrent.futures
from geopy.distance import geodesic
from math import radians, sin, cos, atan2, sqrt
from itertools import combinations
from shapely.geometry import box, Point, Polygon
from ast import literal_eval

In [39]:
def fetch_nearby_buildings_by_count(lat, lon, max_nodes=2500, output_file='Sydney_Buildings.csv'):
    overpass_url = "http://overpass-api.de/api/interpreter"

    overpass_query = f"""
    [out:json][timeout:60];
    (
      way["building"="apartments"](around:1000,{lat},{lon});
      way["amenity"="restaurant"](around:1500,{lat},{lon});
      way["amenity"="cafe"](around:1500,{lat},{lon});
      node["shop"="supermarket"](around:800,{lat},{lon})["name"~"Woolworths|Coles"];
    );
    out center {max_nodes};
    """

    try:
        response = requests.get(overpass_url, params={'data': overpass_query}, timeout=90)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return

    data = response.json()
    elements = data.get("elements", [])
    if not elements:
        print("No nearby buildings found.")
        return

    print(f"✅ Retrieved {min(len(elements), max_nodes)} raw elements near ({lat}, {lon})")

    # Counters for each type
    type_counts = {
        "apartments": 0,
        "restaurant": 0,
        "cafe": 0,
        "supermarket": 0
    }

    with open(output_file, mode='w', newline='', encoding='utf-8-sig') as file:
        fieldnames = ["Node_ID", "Node", "Type", "Latitude", "Longitude", "GPS_coordinates", "Estimated_Height"]
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()

        node_id_counter = 1

        for element in elements:
            tags = element.get('tags', {})
            name = tags.get('name', '').strip()
            if not name:
                continue

            lat_val = element.get('center', {}).get('lat') or element.get('lat')
            lon_val = element.get('center', {}).get('lon') or element.get('lon')
            if lat_val is None or lon_val is None:
                continue

            element_type = (
                tags.get('amenity') or
                tags.get('shop') or
                tags.get('building') or
                'Unknown'
            )

            if element_type not in type_counts:
                continue

            # For apartments only, estimate and filter by height
            if element_type == "apartments":
                height = None
                if 'height' in tags:
                    try:
                        height = float(tags['height'])
                    except ValueError:
                        height = None
                elif 'building:levels' in tags:
                    try:
                        levels = float(tags['building:levels'])
                        height = levels * 3  # Approx. 3m per level
                    except ValueError:
                        height = None

                if height is None or height <= 30 or height >= 120:
                    continue  # skip apartments that don't meet height condition
            else:
                height = None  # not required for other types

            writer.writerow({
                "Node_ID": f"Node_{node_id_counter}",
                "Node": name,
                "Type": element_type,
                "Latitude": lat_val,
                "Longitude": lon_val,
                "GPS_coordinates": f"{lat_val}, {lon_val}",
                "Estimated_Height": height
            })

            type_counts[element_type] += 1
            node_id_counter += 1

            if node_id_counter > max_nodes:
                break

    print(f"📄 Output written to '{output_file}'")
    print(f"✅ Total nodes saved: {sum(type_counts.values())}")

# --- Run the function ---
fetch_nearby_buildings_by_count(
    lat=-33.873245725325454,
    lon=151.20711588465713,
    max_nodes=2500,
    output_file='Sydney_Buildings.csv'
)

✅ Retrieved 187 raw elements near (-33.873245725325454, 151.20711588465713)
📄 Output written to 'Sydney_Buildings.csv'
✅ Total nodes saved: 87


In [40]:
# Cleaning dataset- removing duplicates
df = pd.read_csv("Sydney_Buildings.csv")  # replace with your actual CSV file

# Drop duplicate rows based on the GPS_coordinates column
df_unique = df.drop_duplicates(subset=['GPS_coordinates'])
df_unique.to_csv("Sydney_Buildings.csv", index=False)

In [41]:
# Cleaning dataset- removing Unnamed nodes
df = pd.read_csv("Sydney_Buildings.csv")

# Drop rows where Node_Name is "Unnamed" (case-insensitive, leading/trailing spaces removed)
df_cleaned = df[~df['Node'].str.strip().str.lower().eq('unnamed')]
df_cleaned.to_csv("Sydney_Buildings.csv", index=False)

In [42]:
# Function to query Overpass for building height
def fetch_building_height(lat, lon, radius=10):
    overpass_url = "http://overpass-api.de/api/interpreter"
    query = f"""
    [out:json][timeout:25];
    (
      way["building"](around:{radius},{lat},{lon});
      node["building"](around:{radius},{lat},{lon});
    );
    out body;
    """
    try:
        response = requests.get(overpass_url, params={'data': query})
        response.raise_for_status()
        data = response.json()
        for element in data.get('elements', []):
            tags = element.get('tags', {})
            if 'height' in tags:
                try:
                    return float(tags['height'])
                except ValueError:
                    pass  # Non-numeric value, skip
            elif 'building:levels' in tags:
                try:
                    return float(tags['building:levels']) * 3  # Estimate: 3 meters per level
                except ValueError:
                    pass  # Non-numeric levels, skip
    except Exception as e:
        print(f"Error fetching for ({lat}, {lon}): {e}")
    return None

# Function to process a single row
def process_row(i_row):
    i, row = i_row
    lat, lon = row['Latitude'], row['Longitude']
    height = fetch_building_height(lat, lon)
    if height is None:
        btype = row['Type'].strip().lower() if isinstance(row['Type'], str) else ''
        if btype != 'apartments':
            height = 80
    return i, height

df = pd.read_csv("Sydney_Buildings.csv")
df['Building_Height'] = None

# Run parallel processing
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    for i, height in executor.map(process_row, df.iterrows()):
        df.at[i, 'Building_Height'] = height

df.to_csv("Sydney_Buildings.csv", index=False)


In [43]:
df = pd.read_csv("Sydney_Buildings.csv")

#cleaning dataset-removing buildings with no height fetched
# Replace empty strings with NaN if any
df['Building_Height'].replace('', pd.NA, inplace=True)

# Drop rows where Building_Height is NaN
df_cleaned = df.dropna(subset=['Building_Height'])
df_cleaned.to_csv("Sydney_Buildings.csv", index=False)


In [44]:
# Check for obstructing nodes for generating LoS-based skyway segments
def haversine_distance(coord1, coord2):
    lat1, lon1 = map(float, coord1.split(','))
    lat2, lon2 = map(float, coord2.split(','))
    R = 6371.0  
    lat1_rad, lon1_rad = radians(lat1), radians(lon1)
    lat2_rad, lon2_rad = radians(lat2), radians(lon2)
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

def find_obstructing_nodes(node1_idx, node2_idx, nodes_data, distance_threshold=0.001):
    obstructing_nodes = []
    
    gps1 = nodes_data.at[node1_idx, 'GPS_coordinates']
    gps2 = nodes_data.at[node2_idx, 'GPS_coordinates']
    height1 = nodes_data.at[node1_idx, 'Building_Height']
    height2 = nodes_data.at[node2_idx, 'Building_Height']
    total_distance = haversine_distance(gps1, gps2)
    max_height = max(height1, height2)

    for idx, row in nodes_data.iterrows():
        if idx != node1_idx and idx != node2_idx:
            gps_ob = row['GPS_coordinates']
            height_ob = row['Building_Height']
            if height_ob > max_height:
                d1 = haversine_distance(gps1, gps_ob)
                d2 = haversine_distance(gps2, gps_ob)
                if abs(d1 + d2 - total_distance) < distance_threshold:
                    obstructing_nodes.append({
                        'Node_Pair': f'{node1_idx}-{node2_idx}',
                        'Node1': nodes_data.at[node1_idx, 'Node'],
                        'Node1_Type': nodes_data.at[node1_idx, 'Type'],
                        'Node2': nodes_data.at[node2_idx, 'Node'],
                        'Node2_Type': nodes_data.at[node2_idx, 'Type'],
                        'Obstructing_Nodes': row['Node'],
                        'GPS_Coordinates_Node1': gps1,
                        'GPS_Coordinates_Node2': gps2,
                        'GPS_Coordinates_Obstruction': gps_ob,
                        'Height_Node1': height1,
                        'Height_Node2': height2,
                        'Height_Obstruction': height_ob
                    })

    if not obstructing_nodes:
        obstructing_nodes.append({
            'Node_Pair': f'{node1_idx}-{node2_idx}',
            'Node1': nodes_data.at[node1_idx, 'Node'],
            'Node1_Type': nodes_data.at[node1_idx, 'Type'],
            'Node2': nodes_data.at[node2_idx, 'Node'],
            'Node2_Type': nodes_data.at[node2_idx, 'Type'],
            'Obstructing_Nodes': None,
            'GPS_Coordinates_Node1': gps1,
            'GPS_Coordinates_Node2': gps2,
            'GPS_Coordinates_Obstruction': None,
            'Height_Node1': height1,
            'Height_Node2': height2,
            'Height_Obstruction': None
        })
    return obstructing_nodes

def generate_obstruction_csv(nodes_data):
    result_data = []
    node_combinations = combinations(nodes_data.index, 2)  # For Unique pairs of nodes
    for node1_idx, node2_idx in node_combinations:
        result_data.extend(find_obstructing_nodes(node1_idx, node2_idx, nodes_data))
    pd.DataFrame(result_data).to_csv('obstruction_data_check.csv', index=False)


building_data = pd.read_csv('Sydney_Buildings.csv', encoding='utf-8-sig')
generate_obstruction_csv(building_data)

In [45]:
obstruction_data = pd.read_csv('obstruction_data_check.csv')

# Function to calculate Haversine distance between two GPS coordinates
def haversine_distance(coord1, coord2):
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1, lon1 = map(radians, map(float, coord1.split(', ')))
    lat2, lon2 = map(radians, map(float, coord2.split(', ')))
    # Calculate the differences in coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    # Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    # Calculate the distance
    distance = R * c
    return distance

obstruction_data['Edge_Length (Km)'] = obstruction_data.apply(lambda row: haversine_distance(row['GPS_Coordinates_Node1'], row['GPS_Coordinates_Node2']), axis=1)
obstruction_data.to_csv('obstruction_data_check.csv', index=False)

In [46]:
obstruction_data = pd.read_csv('obstruction_data_check.csv')

# Create an empty graph
graph = nx.DiGraph()

# Iterate through the rows of the dataset to add nodes and edges to the graph
for index, row in obstruction_data.iterrows():
    node1 = row['Node1']
    node2 = row['Node2']
    obstructing_nodes = row['Obstructing_Nodes']
    edge_length = row.get('Edge_Length (Km)', 1.0) 
    if pd.isna(obstructing_nodes) and edge_length < 2.0:
        graph.add_edge(node1, node2, length=edge_length)

first_coord = obstruction_data['GPS_Coordinates_Node1'].dropna().iloc[0]
map_center = [float(coord) for coord in first_coord.split(', ')]

# Create the Folium map
mymap = folium.Map(location=map_center, zoom_start=15)
title_html = """
    <h3 align="center" style="font-size:16px"><b>Skyway Network of Sydney CBD</b></h3>
    """
mymap.get_root().html.add_child(folium.Element(title_html))

# For adding nodes to the map
for node in graph.nodes():
    row = obstruction_data[obstruction_data['Node1'] == node]
    if not row.empty:
        gps_coordinates = [float(coord) for coord in row['GPS_Coordinates_Node1'].iloc[0].split(', ')]
        folium.Marker(gps_coordinates, popup=f'{node}').add_to(mymap)

# For adding edges to the map
for edge in graph.edges(data=True):
    row1 = obstruction_data[obstruction_data['Node1'] == edge[0]]
    row2 = obstruction_data[obstruction_data['Node2'] == edge[1]]

    if not row1.empty and not row2.empty:
        coordinates_node1 = [float(coord) for coord in row1['GPS_Coordinates_Node1'].iloc[0].split(', ')]
        coordinates_node2 = [float(coord) for coord in row2['GPS_Coordinates_Node2'].iloc[0].split(', ')]

        edge_info = f"{edge[0]}_{edge[1]}_{edge[2]['length']}"
        edge_color = "#{:06x}".format(hash(edge_info) & 0xFFFFFF)
        edge_label = f"Edge Length: {edge[2]['length']:.2f} km"

        folium.PolyLine(
            [coordinates_node1, coordinates_node2],
            color=edge_color,
            weight=2.5,
            opacity=1,
            popup=edge_label
        ).add_to(mymap)

# Skyway Map with LoS-based skyway Segments and Nodes
mymap.save('skyway_network_cbd.html')

In [47]:
# Extract node names, segment length, building (i.e., nodes) heights, and node types from the graph
nodes_data = []
for node in graph.nodes(data=True):
    node_id = node[0]

    row = obstruction_data[obstruction_data['Node1'] == node_id]
    if not row.empty:
        gps = row['GPS_Coordinates_Node1'].iloc[0]
        height = row['Height_Node1'].iloc[0] 
        btype = row['Node1_Type'].iloc[0]
    else:
        row = obstruction_data[obstruction_data['Node2'] == node_id]
        if not row.empty:
            gps = row['GPS_Coordinates_Node2'].iloc[0]
            height = row['Height_Node2'].iloc[0]  
            btype = row['Node2_Type'].iloc[0]
        else:
            continue

    node_data = {
        'Node': node_id,
        'GPS_Coordinates': gps,
        'Building_Height': height,
        'Building_Type': btype
    }
    nodes_data.append(node_data)

# Extract edges/skyway segments
edges_data = []
for edge in graph.edges(data=True):
    edge_data = {
        'Node1': edge[0],
        'Node2': edge[1],
        'Edge_Length(Km)': edge[2]['length'],
    }
    edges_data.append(edge_data)


nodes_df = pd.DataFrame(nodes_data)
edges_df = pd.DataFrame(edges_data)

result_df = pd.merge(edges_df, nodes_df, left_on='Node1', right_on='Node', how='left')
result_df.rename(columns={
    'GPS_Coordinates': 'GPS_Coordinates_Node1', 
    'Building_Type': 'Building_Type_Node1', 
    'Building_Height': 'Building_Height_Node1'
}, inplace=True)
result_df = pd.merge(result_df, nodes_df, left_on='Node2', right_on='Node', how='left', suffixes=('', '_Node2'))
result_df.rename(columns={
    'GPS_Coordinates': 'GPS_Coordinates_Node2', 
    'Building_Type': 'Building_Type_Node2', 
    'Building_Height': 'Building_Height_Node2'
}, inplace=True)

# Drop unnecessary columns
result_df.drop(['Node', 'Node_Node2'], axis=1, inplace=True)
result_df.to_csv('Skyway_network_data_check.csv', index=False)

In [48]:
file_path = 'Skyway_network_data_check.csv'
skyway_df = pd.read_csv(file_path)

# Function to calculate the geodesic distance in kilometers
def calculate_distance(row):
    # Parse the GPS coordinates
    node1_coords = tuple(map(float, row['GPS_Coordinates_Node1'].split(', ')))
    node2_coords = tuple(map(float, row['GPS_Coordinates_Node2'].split(', ')))
    # Calculate the distance
    return geodesic(node1_coords, node2_coords).kilometers
skyway_df['Edge_Length(Km)'] = skyway_df.apply(calculate_distance, axis=1)

updated_file_path = 'Skyway_network_data_check.csv'
skyway_df.to_csv(updated_file_path, index=False)

In [49]:
input_file = 'Skyway_network_data_check.csv'
skyway_data = pd.read_csv(input_file)

# Add a unique Segment ID to each edge
skyway_data['Segment_ID'] = ['SEG' + str(i+1) for i in range(len(skyway_data))]
output_file = 'Skyway_network_data_check.csv'
skyway_data.to_csv(output_file, index=False)

In [50]:
skyway_data = pd.read_csv('Skyway_network_data_check.csv')

# Create a set of unique node IDs
unique_nodes = pd.unique(skyway_data[['Node1', 'Node2']].values.ravel())

# Create a mapping of nodes to unique IDs
node_id_map = {node: f'Node_{i+1}' for i, node in enumerate(unique_nodes)}

# Add unique Node IDs to the original DataFrame
skyway_data['Node1_ID'] = skyway_data['Node1'].map(node_id_map)
skyway_data['Node2_ID'] = skyway_data['Node2'].map(node_id_map)

output_file = 'Skyway_network_data_check.csv'
skyway_data.to_csv(output_file, index=False)

In [51]:
skyway_data = pd.read_csv('Skyway_network_data_check.csv')

# Create a new column that represents the undirected edge (a sorted tuple of Node1_ID and Node2_ID)
skyway_data['Edge_Key'] = skyway_data.apply(lambda row: tuple(sorted([row['Node1_ID'], row['Node2_ID']])), axis=1)

# Create a dictionary to ensure consistent Segment_ID assignment
segment_id_map = {}

for index, row in skyway_data.iterrows():
    edge_key = row['Edge_Key']
    if edge_key not in segment_id_map:
        # Assign the current Segment_ID if not already assigned
        segment_id_map[edge_key] = row['Segment_ID']
    else:
        # Update the Segment_ID in the current row for consistency
        skyway_data.at[index, 'Segment_ID'] = segment_id_map[edge_key]

# Drop the temporary Edge_Key column as it's no longer needed
skyway_data = skyway_data.drop(columns=['Edge_Key'])

output_file = 'Skyway_network_data_check.csv'
skyway_data.to_csv(output_file, index=False)

In [52]:
input_file = "Skyway_network_data_check.csv"
df = pd.read_csv(input_file)

# Drop rows where Node1 and Node2 are the same
df_filtered = df[df['Node1'] != df['Node2']]
output_file = "Skyway_network_data_check.csv"
df_filtered.to_csv(output_file, index=False)

In [53]:
input_file = 'Skyway_network_data_check.csv'
backup_file = 'Skyway_network_data_backupFile.csv'
output_file = 'Skyway_network_data_checkF.csv'

df = pd.read_csv(input_file)
backup_df = df.copy()
backup_df.to_csv(backup_file, index=False)

# Drop rows where building height is greater than 120 meters (400 ft) as per CASA regulations
df = df[(df['Building_Height_Node1'] <= 120) &
        (df['Building_Height_Node2'] <= 120) ]
df.to_csv(output_file, index=False)

In [54]:
input_file = 'Skyway_network_data_checkF.csv'
df = pd.read_csv(input_file)

# Define segment dimensions as per the dimensions of DJI phantom 3
incoming_lane_height = 0.3  # meters
separation_between_lanes = 0.1  # meters
outgoing_lane_height = 0.3  # meters

# Total height = incoming + separation + outgoing
total_segment_height = incoming_lane_height + separation_between_lanes + outgoing_lane_height
segment_width = 0.3  # meters (fixed for all segments)

df['Incoming_Lane_Height'] = incoming_lane_height
df['Outgoing_Lane_Height'] = outgoing_lane_height
df['Lane_Separation'] = separation_between_lanes
df['Segment_Height'] = total_segment_height
df['Segment_Width'] = segment_width

df.rename(columns={'Edge_Length(Km)': 'Segment_Length'}, inplace=True)
df.to_csv(input_file, index=False)

In [55]:
input_file = "Skyway_network_data_checkF.csv"
df = pd.read_csv(input_file)

# Function to convert GPS coordinates to XYZ coordinates
def gps_to_xyz(lat, lon, altitude):
    # Radius of Earth in meters
    R = 6371000
    lat_rad = np.radians(lat)
    lon_rad = np.radians(lon)
    
    x = (R + altitude) * np.cos(lat_rad) * np.cos(lon_rad)
    y = (R + altitude) * np.cos(lat_rad) * np.sin(lon_rad)
    z = (R + altitude) * np.sin(lat_rad)
    return x, y, z

# Function to generate cuboid geometry for each segment
def generate_geometry(row):
    # Parse GPS coordinates
    lat1, lon1 = map(float, row["GPS_Coordinates_Node1"].split(", "))
    lat2, lon2 = map(float, row["GPS_Coordinates_Node2"].split(", "))
    # Determine the maximum height between the two nodes
    max_height = max(row["Building_Height_Node1"], row["Building_Height_Node2"])
    # Segment height remains the same
    segment_height = row["Segment_Height"]
    # Convert GPS coordinates and height to XYZ coordinates using the maximum height
    x1, y1, z1 = gps_to_xyz(lat1, lon1, max_height)
    x2, y2, z2 = gps_to_xyz(lat2, lon2, max_height)
    # Ensure Z-coordinates consistency for reversed rows
    z_adjusted = max_height + (segment_height / 2)
    z1, z2 = z_adjusted, z_adjusted

    # Create a geometry dictionary
    geometry = {
        "start": {"x": x1, "y": y1, "z": z1},
        "end": {"x": x2, "y": y2, "z": z2},
        "width": row["Segment_Width"],
        "height": segment_height,
    }
    return geometry

# Apply the function to each row and create a new column for geometry
df["Segment_Geometry"] = df.apply(generate_geometry, axis=1)
output_file = "Skyway_network_data_checkF.csv"
df.to_csv(output_file, index=False)

In [56]:
input_file = "Skyway_network_data_checkF.csv"
df = pd.read_csv(input_file)

# Function to calculate Euclidean distance between two 3D points
def euclidean_distance(point1, point2):
    return sqrt((point1["x"] - point2["x"])**2 + (point1["y"] - point2["y"])**2 + (point1["z"] - point2["z"])**2)

# Function to generate a 3D bounding box (cuboid) for a segment
def generate_cuboid(segment):
    start = segment["start"]
    end = segment["end"]
    width = segment["width"]
    height = segment["height"]
    x_min, x_max = min(start["x"], end["x"]), max(start["x"], end["x"])
    y_min, y_max = min(start["y"], end["y"]), max(start["y"], end["y"])
    z_min, z_max = min(start["z"], end["z"]), max(start["z"], end["z"])
    x_min -= width / 2
    x_max += width / 2
    z_max += height
    
    return box(x_min, y_min, x_max, y_max), z_min, z_max, start, end

# Function to calculate 3D volume of an intersection
def calculate_intersection_volume(intersection, z_min1, z_max1, z_min2, z_max2):
    if not isinstance(intersection, Polygon):
        return 0
    z_overlap = max(0, min(z_max1, z_max2) - max(z_min1, z_min2))
    return intersection.area * z_overlap

# Function to check intersection of two cuboids
def check_intersection(cuboid1, cuboid2, z_min1, z_max1, z_min2, z_max2, start1, end1, start2, end2):
    intersection_2d = cuboid1.intersection(cuboid2)
    if intersection_2d.is_empty:
        return False
    if z_max1 <= z_min2 or z_max2 <= z_min1:
        return False
    volume = calculate_intersection_volume(intersection_2d, z_min1, z_max1, z_min2, z_max2)
    if (
        euclidean_distance(start1, start2) <= 6 or euclidean_distance(start1, end2) <= 6 or
        euclidean_distance(end1, start2) <= 6 or euclidean_distance(end1, end2) <= 6
    ) and volume > 0.3:
        return False
    return True


# Pre-parse geometry column
df["Segment_Geometry_Parsed"] = df["Segment_Geometry"].apply(literal_eval)
intersections = []
for i in range(len(df)):
    row1 = df.iloc[i]
    geom1 = row1["Segment_Geometry_Parsed"]
    cuboid1, z_min1, z_max1, start1, end1 = generate_cuboid(geom1)

    for j in range(i + 1, len(df)):
        row2 = df.iloc[j]
        geom2 = row2["Segment_Geometry_Parsed"]

        # Skip reverse segments
        if {row1["Node1"], row1["Node2"]} == {row2["Node1"], row2["Node2"]}:
            continue

        cuboid2, z_min2, z_max2, start2, end2 = generate_cuboid(geom2)
        if check_intersection(cuboid1, cuboid2, z_min1, z_max1, z_min2, z_max2, start1, end1, start2, end2):
            intersection_point = cuboid1.intersection(cuboid2)
            intersections.append({
                "Segment1_ID": row1["Segment_ID"],
                "Segment2_ID": row2["Segment_ID"],
                "Intersection_Point": intersection_point.bounds
            })

intersection_df = pd.DataFrame(intersections)
intersection_df.to_csv('check.csv', index=False)

In [57]:
# Create a set of intersecting segments
intersecting_segments = set()
for _, row in intersection_df.iterrows():
    intersecting_segments.add(row["Segment1_ID"])
    intersecting_segments.add(row["Segment2_ID"])

retain_segments = set()
drop_segments = []

for _, row in intersection_df.iterrows():
    seg1 = row["Segment1_ID"]
    seg2 = row["Segment2_ID"]
    length1 = df[df["Segment_ID"] == seg1]["Segment_Length"].values[0]
    length2 = df[df["Segment_ID"] == seg2]["Segment_Length"].values[0]
    
    if length1 >= length2:
        retain_segments.add(seg1)
        drop_segments.append(seg2)
    else:
        retain_segments.add(seg2)
        drop_segments.append(seg1)

# Retain non-intersecting segments
non_intersecting_segments = set(df["Segment_ID"]) - intersecting_segments
retain_segments.update(non_intersecting_segments)

# Filter the original DataFrame
retained_df = df[df["Segment_ID"].isin(retain_segments)]
dropped_segments_df = df[df["Segment_ID"].isin(drop_segments)]
retained_df.to_csv("Skyway_network_data_retained.csv", index=False)
dropped_segments_df.to_csv("Skyway_network_data_dropped.csv", index=False)

In [58]:
df = pd.read_csv("Skyway_network_data_retained.csv")

# Add new columns for rooftop areas
df["Rooftop_Area_Node1"] = 0.0
df["Rooftop_Area_Node2"] = 0.0
# Dictionary for caching computed areas
area_cache = {}

# Function to fetch rooftop area by GPS cooridnates + node names
def fetch_rooftop_area(lat, lon, expected_name):
    # Create a key combining GPS and node name (cleaned)
    coord_key = f"{lat:.7f},{lon:.7f}-{expected_name.strip().lower()}"
    if coord_key in area_cache:
        return area_cache[coord_key]

    def try_fetch(dist):
        tags = {"building": True}
        gdf = ox.features_from_point(center_point=(lat, lon), tags=tags, dist=dist)
        buildings = gdf[gdf.geometry.type == "Polygon"]
        if buildings.empty:
            return None

        buildings_proj = buildings.to_crs(buildings.estimate_utm_crs())
        if 'name' in buildings_proj.columns:
            buildings_proj['name'] = buildings_proj['name'].astype(str).str.strip().str.lower()
            expected_clean = expected_name.strip().lower()
            match = buildings_proj[buildings_proj['name'] == expected_clean]
            if not match.empty:
                return match.geometry.area.min()
        return buildings_proj.geometry.area.min()
    try:
        area = try_fetch(40)
        if area is None:
            area = try_fetch(50)
        result = area if area is not None else 0.0
        area_cache[coord_key] = result
        return result
    except Exception as e:
        print(f"Error at ({lat}, {lon}) for '{expected_name}': {e}")
        area_cache[coord_key] = 0.0
        return 0.0

# Loop through each row and compute rooftop areas
for idx, row in df.iterrows():
    try:
        lat1, lon1 = map(float, row["GPS_Coordinates_Node1"].split(","))
        lat2, lon2 = map(float, row["GPS_Coordinates_Node2"].split(","))
        name1 = str(row["Node1"])
        name2 = str(row["Node2"])

        df.at[idx, "Rooftop_Area_Node1"] = fetch_rooftop_area(lat1, lon1, name1)
        df.at[idx, "Rooftop_Area_Node2"] = fetch_rooftop_area(lat2, lon2, name2)
    except Exception as e:
        print(f"Skipping row {idx}: {e}")

df.to_csv("Skyway_network_data_retained.csv", index=False)

In [59]:
df = pd.read_csv("Skyway_network_data_retained.csv")

# Rooftop areas reflect only 1.31%-11.6% (take average=6.4%=0.064) usable space
df["Usable_Rooftop_Area_Node1(m^2)"] = df["Rooftop_Area_Node1"] * 0.064
df["Usable_Rooftop_Area_Node2(m^2)"] = df["Rooftop_Area_Node2"] * 0.064
df.to_csv("Skyway_network_data_retained.csv", index=False)

In [60]:
input_file = 'Skyway_network_data_retained.csv'
df = pd.read_csv(input_file)

# Pad dimensions and spacing
PAD_WIDTH = 1.35  # meters
SPACING = 1.0     # meters

# Effective area per pad with spacing
EFFECTIVE_PAD_AREA_M2 = (PAD_WIDTH + SPACING) ** 2
MIN_PAD_COUNT = 2

df = df[
    (df['Usable_Rooftop_Area_Node1(m^2)'] <= 100) &
    (df['Usable_Rooftop_Area_Node2(m^2)'] <= 100)
]
#Function to compute the number of charging and waiting pads per node
def compute_half_pad_pairs(rooftop_area):
    try:
        half_area = rooftop_area / 2
        pad_count = math.floor(half_area / EFFECTIVE_PAD_AREA_M2)
        return max(pad_count, MIN_PAD_COUNT)
    except Exception as e:
        return MIN_PAD_COUNT

df['Recharging_Pads_Node1'] = df['Usable_Rooftop_Area_Node1(m^2)'].apply(compute_half_pad_pairs)
df['Waiting_Pads_Node1'] = df['Usable_Rooftop_Area_Node1(m^2)'].apply(compute_half_pad_pairs)
df['Recharging_Pads_Node2'] = df['Usable_Rooftop_Area_Node2(m^2)'].apply(compute_half_pad_pairs)
df['Waiting_Pads_Node2'] = df['Usable_Rooftop_Area_Node2(m^2)'].apply(compute_half_pad_pairs)

output_file = 'Skyway_network_data_retained.csv'
df.to_csv(output_file, index=False)

In [62]:
skyway_data = pd.read_csv('Skyway_network_data_retained.csv')

# Collect valid source-destination pairs
new_data = []

for index, row in skyway_data.iterrows():
    # Case 1: Node1 is a destination
    if row['Building_Type_Node1'] in ['apartments', 'hospital'] and row['Building_Type_Node2'] not in ['apartments', 'hospital']:
        new_data.append({
            'Source_Node': row['Node2'],
            'Source_Type': row['Building_Type_Node2'],
            'Source_Coordinates': row['GPS_Coordinates_Node2'],
            'Destination_Node': row['Node1'],
            'Destination_Type': row['Building_Type_Node1'],
            'Destination_Coordinates': row['GPS_Coordinates_Node1'],
            'Building_Height': row['Building_Height_Node1'],
            'Type': row['Building_Type_Node1']
        })

    # Case 2: Node2 is a destination
    if row['Building_Type_Node2'] in ['apartments', 'hospital'] and row['Building_Type_Node1'] not in ['apartments', 'hospital']:
        new_data.append({
            'Source_Node': row['Node1'],
            'Source_Type': row['Building_Type_Node1'],
            'Source_Coordinates': row['GPS_Coordinates_Node1'],
            'Destination_Node': row['Node2'],
            'Destination_Type': row['Building_Type_Node2'],
            'Destination_Coordinates': row['GPS_Coordinates_Node2'],
            'Building_Height': row['Building_Height_Node2'],
            'Type': row['Building_Type_Node2']
        })

result_df = pd.DataFrame(new_data)
result_df = result_df[result_df['Type'] != 'hospital']
result_df.to_csv('Skyway_Source_Destination.csv', index=False)